# Visualization of test images with a given input model

Given a path to the trained UNet model, the script visualises 5 test images predicted by the trained model

In [ ]:
#libraries

import torch
import models
import os
import yaml
import numpy as np
from PIL import Image
from patchify import patchify, unpatchify
import matplotlib.pyplot as plt
from skimage.exposure import equalize_hist
import math
from skimage.metrics import structural_similarity as ssim
from torchvision.transforms import Resize
import torch.nn as nn
import cv2

In [ ]:
# Original
def image_transform(img, amp=False, raw=False, output=False, crop = True):
    if raw and not output:
        arr = np.fromfile(open(img, 'rb'), dtype=np.uint8).reshape(2160,4096)
        arr = np.repeat(np.expand_dims(arr, axis=2), 3, axis=2)
    else:
        img = cv2.imread(img, -1)

    if crop:
        # print(img.shape, 'img shape before cropping')
        img = img[:2048, :, :]
    # if output:
        # print(arr, 'Output array printed')
    if output == False:
        # print('Input')
        img = torch.from_numpy(img / 255).permute(2, 0, 1).unsqueeze(0)
        if amp:
        # print(arr.shape, 'Shape in here')
        # amp = amp_module(arr, 0.5)
            img = (img*amp).clamp_(0,1)
        # arr = ()
    else:
        # print('Output')
        img = torch.from_numpy(img/255).permute(2, 0, 1).unsqueeze(0)
    # res, h, w, p_x, p_y = patchify_img(np.array(arr), patch_size=512)
    # arr = torch.from_numpy(res).permute(0, 3, 1, 2)
    # if amp:
        # return arr, h, w, p_x, p_y, amp, amp_arr
    # else:
        # return arr, h, w, p_x, p_y
    # print(arr.shape, 'arr shape')
    # print(img.shape, 'img.shape')
    return img
    

In [ ]:
device=torch.device('cuda:1')
torch.cuda.device_count()
dir = '/home/gpu/girish/results_clrimg_resized/_unet_basic2_100_new_amp_100'
amp = 100
split_ratio = 0
raw = False
monochrome= False
crop = True

In [ ]:
#model construction
model_path = os.path.join(dir, 'epoch-best-psnr.pth')
sv_file = torch.load(model_path, map_location=device)
model = models.make(sv_file['model'], load_sd=True).eval().cuda()
model = nn.DataParallel(model, device_ids=[1])

In [ ]:
# model_weights_path = os.path.join(dir, "epoch-best-psnr.pth")
# sv_file = torch.load(model_weights_path)
# model = models.make(sv_file["model"], load_sd=True).cuda()


In [ ]:
# #! Patichyfies ? 

# def patchify_img(image, patch_size=256):
#     size_x = (image.shape[0] // patch_size) * patch_size  # get width to nearest size divisible by patch size
#     size_y = (image.shape[1] // patch_size) * patch_size
#     instances = []

#     # Crop original image to size divisible by patch size from top left corner
#     image = image[:size_x, :size_y, :]

#     # Extract patches from each image, step=patch_size means no overlap
#     patch_img = patchify(image, (patch_size, patch_size, 3), step=patch_size)

#     # iterate over vertical patch axis
#     for j in range(patch_img.shape[0]):
#         # iterate over horizontal patch axis
#         for k in range(patch_img.shape[1]):
#             # patches are located like a grid. use (j, k) indices to extract single patched image
#             single_patch_img = patch_img[j, k]

#             # Drop extra extra dimension from patchify
#             instances.append(np.squeeze(single_patch_img))
#     return np.vstack([np.expand_dims(x, 0) for x in instances]), size_x, size_y, patch_img.shape[0], patch_img.shape[1]

In [ ]:

# def amp_module(img, m):
#   """
#   img.shape - (3, H, W)
#   m - central value (typically 0.5) 
#   """
#   # H = img.shape[0]
#   W = img.shape[1]
#   sumOfPixels = torch.sum(img)
#   amp = m*(3*H*W)/(sumOfPixels)
#   print(amp)
#   return amp


In [ ]:
# def image_transform(img, amp=False, raw=False, output=False):
#     if raw and not output:
#         arr = np.fromfile(open(img, 'rb'), dtype=np.uint8).reshape(2160,4096)
#         arr = np.repeat(np.expand_dims(arr, axis=2), 3, axis=2)
#     else:
#         arr = np.array(Image.open(img))
#         if len(arr.shape)==3:
#             pass
#         elif len(arr.shape)==2:
#             arr = np.repeat(np.expand_dims(arr, axis=2), 3, axis=2)
#     res, h, w, p_x, p_y = patchify_img(np.array(arr), patch_size=512)
#     arr = torch.from_numpy(res/255).permute(0, 3, 1, 2)
#     if amp:
#         arr = (arr*amp).clamp_(0,1)
#     return arr, h, w, p_x, p_y
    

In [ ]:
# #load image - Probably the main function everyhting seems to happen - 18secs to run

# # Original

# # Train Config is being loaded here 
# with open(os.path.join(dir, 'config.yaml'), 'r') as f:
#         config = yaml.load(f, Loader=yaml.FullLoader)
        
# if config['val_dataset']['dataset']['args'].get('root_path_inp') is not None:
#     test_dir = config['val_dataset']['dataset']['args']['root_path_inp']
#     out_dir = config['val_dataset']['dataset']['args']['root_path_out']
#     filenames = sorted(os.listdir(test_dir))
#     outfile = sorted(os.listdir(out_dir))
#     img_files = filenames[math.ceil(len(filenames)*split_ratio):]
#     print(len(img_files), len(filenames))
#     #img_files = filenames[0:]
#     imgs, h_s, w_s, pxs, pys, out_imgs = [], [], [], [], [], []
#     for file in img_files:
#         res = image_transform(os.path.join(test_dir,file), amp, raw)
#         if monochrome:
#             imgs.append(res[0][:,:1,:,:])
#         else:
#             imgs.append(res[0])
#         h_s.append(res[1])
#         w_s.append(res[2])
#         pxs.append(res[3])
#         pys.append(res[4])
#         out_imgs.append(image_transform(os.path.join(out_dir, file), raw=raw, output=True)[0])
                       
# else:
#     test_dir1 = config['val_dataset']['dataset']['args']['root_path_inp1']
#     test_dir2 = config['val_dataset']['dataset']['args']['root_path_inp2']
#     out_dir = config['val_dataset']['dataset']['args']['root_path_out']
#     filenames = sorted(os.listdir(test_dir1))
#     img_files = filenames[math.ceil(len(filenames)*split_ratio):]
#     outfile = sorted(os.listdir(out_dir))
#     imgs, h_s, w_s, pxs, pys, out_imgs = [], [], [], [], [], []
    
#     for file in img_files:
#         res1 = image_transform(os.path.join(test_dir1, file), amp, raw)
#         res2 = image_transform(os.path.join(test_dir2, file), amp, raw)
    
#         if monochrome:
#             res = torch.cat([res1[0][:,:1,:,:], res2[0][:,:1,:,:]], axis=1)
#             imgs.append(res)
#         else:
#             res = torch.cat([res1[0], res2[0]], dim=1)
#             imgs.append(res)
#         h_s.append(res2[1])
#         w_s.append(res2[2])
#         pxs.append(res2[3])
#         pys.append(res2[4])
#         out_imgs.append(image_transform(os.path.join(out_dir, file), raw=raw, output=True)[0])


In [ ]:
#load image - Probably the main function everyhting seems to happen - 18secs to run

# This is for the TEST Set

# Train Config is being loaded here 
amp_imgs = []
avg_amp = []
with open(os.path.join(dir, 'test_config.yaml'), 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
if config['test_dataset']['dataset']['args'].get('root_path_inp') is not None:
    test_dir = config['test_dataset']['dataset']['args']['root_path_inp']
    out_dir = config['test_dataset']['dataset']['args']['root_path_out']
    filenames = sorted(os.listdir(test_dir))
    outfile = sorted(os.listdir(out_dir))
    img_files = filenames[math.ceil(len(filenames)*split_ratio):]
    print(len(img_files), len(filenames))
    # img_files = ['68.png', '162.png', '134.png', '173.png', '18.png', '184.png'] #! filenames
    imgs, h_s, w_s, pxs, pys, out_imgs = [], [], [], [], [], []
    for file in img_files[:1]:
        # print('Input Transformation started')
        res = image_transform(os.path.join(test_dir,file), amp = amp, raw = raw, crop = crop, output=False)
        # print(res.shape, 'Image after transformation')
        if monochrome:
            imgs.append(res[0][:,:1,:,:])
        else:
            imgs.append(res)
            print(res.shape, 'Input shape res.shape')
        # h_s.append(res[1])
        # w_s.append(res[2])
        # pxs.append(res[3])
        # pys.append(res[4])
        # if amp:
            # amp_imgs.append(res[6])
            # avg_amp.append(res[5])
        # out_imgs.append(image_transform(os.path.join(out_dir, file), raw=raw, output=True)[0])
        # print('Output transformation started')
        out_imgs.append(image_transform(os.path.join(out_dir, file), raw=raw, output=True))
                       
else:
    test_dir1 = config['test_dataset']['dataset']['args']['root_path_inp1']
    test_dir2 = config['test_dataset']['dataset']['args']['root_path_inp2']
    out_dir = config['test_dataset']['dataset']['args']['root_path_out']
    filenames = sorted(os.listdir(test_dir1))
    img_files = filenames[math.ceil(len(filenames)*split_ratio):]
    outfile = sorted(os.listdir(out_dir))
    imgs, h_s, w_s, pxs, pys, out_imgs = [], [], [], [], [], []
    
    for file in img_files:
        res1 = image_transform(os.path.join(test_dir1, file), amp, raw)
        res2 = image_transform(os.path.join(test_dir2, file), amp, raw)
    
        if monochrome:
            res = torch.cat([res1[0][:,:1,:,:], res2[0][:,:1,:,:]], axis=1)
            imgs.append(res)
        else:
            res = torch.cat([res1[0], res2[0]], dim=1)
            imgs.append(res)
        h_s.append(res2[1])
        w_s.append(res2[2])
        pxs.append(res2[3])
        pys.append(res2[4])
        out_imgs.append(image_transform(os.path.join(out_dir, file), raw=raw, output=True)[0])


In [ ]:
# #load image - Probably the main function everyhting seems to happen - 18secs to run



# # Train Config is being loaded here 
# with open(os.path.join(dir, 'config.yaml'), 'r') as f:
#         config = yaml.load(f, Loader=yaml.FullLoader)
        
# if config['val_dataset']['dataset']['args'].get('root_path_inp') is not None:
#     test_dir = config['val_dataset']['dataset']['args']['root_path_inp']
#     out_dir = config['val_dataset']['dataset']['args']['root_path_out']
#     filenames = sorted(os.listdir(test_dir))
#     outfile = sorted(os.listdir(out_dir))
#     img_files = filenames
#     print(len(img_files), len(filenames))
#     #img_files = filenames[0:]
#     imgs, h_s, w_s, pxs, pys, out_imgs = [], [], [], [], [], []
#     for file in img_files:
#         # Image_Transform is happening. Need to check if this is the same transformation happening with testing as well 
#         res = image_transform(os.path.join(test_dir,file), amp, raw)
#         if monochrome:
#             imgs.append(res[0][:,:1,:,:])
#         else:
#             imgs.append(res[0])
#         h_s.append(res[1])
#         w_s.append(res[2])
#         pxs.append(res[3])
#         pys.append(res[4])
#         out_imgs.append(image_transform(os.path.join(out_dir, file), raw=raw, output=True)[0])
                       
# else:
#     test_dir1 = config['val_dataset']['dataset']['args']['root_path_inp1']
#     test_dir2 = config['val_dataset']['dataset']['args']['root_path_inp2']
#     out_dir = config['val_dataset']['dataset']['args']['root_path_out']
#     filenames = sorted(os.listdir(test_dir1))
#     img_files = filenames[math.ceil(len(filenames)*split_ratio):]
#     outfile = sorted(os.listdir(out_dir))
#     imgs, h_s, w_s, pxs, pys, out_imgs = [], [], [], [], [], []
    
#     for file in img_files:
#         res1 = image_transform(os.path.join(test_dir1, file), amp, raw)
#         res2 = image_transform(os.path.join(test_dir2, file), amp, raw)
    
#         if monochrome:
#             res = torch.cat([res1[0][:,:1,:,:], res2[0][:,:1,:,:]], axis=1)
#             imgs.append(res)
#         else:
#             res = torch.cat([res1[0], res2[0]], dim=1)
#             imgs.append(res)
#         h_s.append(res2[1])
#         w_s.append(res2[2])
#         pxs.append(res2[3])
#         pys.append(res2[4])
#         out_imgs.append(image_transform(os.path.join(out_dir, file), raw=raw, output=True)[0])


In [ ]:
def psnr(pred, out, rgb_range=1):
    '''
    inp: patch_count * channels * H * W
    pred: patch_count * channels * H * W
    '''
    diff = (pred - out)/ rgb_range
    mse = torch.mean(torch.pow(diff, 2))

    return -10 * torch.log10(mse)

In [ ]:
len(imgs)

In [ ]:
for i in range(len(img_files)):
    file_name = img_files[i]
    img = imgs[i] # 1 x 3 x 2048 x 4096, torch.float64
    # RESIZING
    img = img.squeeze(dim = 0) # 3 x 2048 x 4096, torch.float64
    resizer = Resize([1024, 2048])
    img = resizer(img) # 3 x 1024 x 2048, torch.float64
    img = img.float() # 3 x 1024 x 2048, torch.float32
    # UNSQUEEZING
    img = img.unsqueeze(dim = 0) # 1 x 3 x 1024 x 2048, torch.float32
    print(img.dtype)
    print(img.shape)
    with torch.no_grad():
        pred = model(((img - 0.5)/0.5))*0.5 + 0.5
    break

In [ ]:
len(out_imgs)
print(out_imgs[0])

In [ ]:
psnr_arr = []
ssim_arr = []
for i in range(len(img_files)):

    # Setting up the image
    resizer = Resize([1024, 2048])
    file_name = img_files[i]

    # Resizing the GT
    gt = out_imgs[i]
    gt = torch.reshape(gt, (3,2048, 4096))
    gt = resizer(gt)


    # Prediction
    img = imgs[i] # 1 x 3 x 2048 x 4096, torch.float64
    # RESIZING
    img = img.squeeze(dim = 0) # 3 x 2048 x 4096, torch.float64
    resizer = Resize([1024, 2048])
    img = resizer(img) # 3 x 1024 x 2048, torch.float64
    img = img.float() # 3 x 1024 x 2048, torch.float32
    # UNSQUEEZING
    img = img.unsqueeze(dim = 0) # 1 x 3 x 1024 x 2048, torch.float32
    print(img.dtype)
    print(img.shape)
    with torch.no_grad():
        pred = model(((img - 0.5)/0.5))*0.5 + 0.5 
    temp = psnr(pred.cpu(), gt).item()
    print(temp)
    psnr_arr.append(temp)


    # Ground truth Display
    plt.figure(figsize=(20,12))
    plt.subplot(2,2,1)
    gt = torch.permute(gt, (1, 2 , 0)).detach().cpu().numpy()
    gt = gt[:, :, [2,1,0]]
    plt.title('Ground Truth {}'.format(file_name))
    plt.imshow(gt, 'gray')
    plt.axis('off')
    plt.subplot(2,2,2)

    pred = torch.permute(pred, (1,2,0)).detach().cpu().numpy
    print(pred)
    plt.title('Output')
    plt.imshow(pred, 'gray')
    plt.axis('off')

    ssim_arr.append(ssim(pred, gt.astype(np.float32()), channel_axis=2))
    plt.subplot(2,2,3)
    inp = torch.permute(inp, (1,2,0)).detach().cpu().numpy()
    plt.title('Amplifid Image')
    plt.imshow(inp, 'gray')

    plt.subplot(2,2,4)
    plt.title('Difference Between Output and Input')
    plt.imshow(abs(gt-pred), 'gray')
    plt.axis('off')

    plt.subplots_adjust(wspace=0.01, hspace=0.1)
    plt.show()



In [ ]:
# #predict
# results, psnrs, ssims = [], [], []
# print(len(imgs))
# for i, img in enumerate(imgs):
#     pred_patches = []
#     # print(img.shape)
#     for patch in img:
#         pred = model((patch.unsqueeze(0).float()-0.5)/0.5)
#         pred_patches.append((pred*0.5+0.5).clamp_(0,1).detach().cpu())
#     pred = torch.vstack(pred_patches)
#     psnrs.append(psnr(pred, out_imgs[i][:,:3,:,:]).item())
#     result = unpatchify(pred.permute(0,2,3,1).reshape(pxs[0],pys[0],1,512,512,3).detach().numpy(), (h_s[0],w_s[0],3))
#     results.append(result)
    

In [ ]:
# gt_ = []
# for i in range(len(results)):
#     if out_imgs[i].shape[0] == 28: #! Kinda try to generalize not plain 28 
#         gt = unpatchify(out_imgs[i][:,:3,:,:].permute(0,2,3,1).reshape(4,7,1,512,512,3).detach().numpy(), (h_s[0],3584,3))
#     else:
#         gt = unpatchify(out_imgs[i][:,:3,:,:].permute(0,2,3,1).reshape(pxs[0],pys[0],1,512,512,3).detach().numpy(), (h_s[0],w_s[0],3))
#     ssims.append(ssim(results[i], gt, channel_axis=2))
#     gt_.append(gt)
    
# avg_ssim = np.mean(ssims)
# print(avg_ssim)

In [ ]:
# #visualise
# sorted_psnrs = sorted(psnrs)

# for psnr in sorted_psnrs[:5]:
#     index = psnrs.index(psnr)
#     plt.figure(figsize=(20,12))
#     plt.subplot(2,2,1)
#     if raw:
#         plt.imshow(np.fromfile(open(os.path.join(out_dir, img_files1[index]), 'rb'), dtype=np.uint8).reshape(2160,4096), 'gray')
#     else:
#         # plt.imshow(Image.open(os.path.join(out_dir, img_files[index])), 'gray')
#         plt.imshow(gt_[index], 'gray')
#     plt.title('Ground Truth - {}'.format(img_files[index]))
#     plt.axis('off')
    
#     plt.subplot(2,2,2)
#     plt.imshow(results[index], 'gray')
#     plt.title('PSNR: '+str(round(psnr,2)) + ' SSIM: '+str(round(ssims[index], 2)))
#     plt.axis('off')
    
#     plt.subplot(2,2,3)
#     # plt.imshow(Image.open(os.path.join(out_dir, img_files[index])), 'gray')
#     plt.imshow(abs(gt_[index] - results[index]),'gray')
#     plt.title('Difference')
#     plt.axis('off')
    
#     plt.subplot(2,2,4)
#     plt.imshow(amp_imgs[index], 'gray')
#     plt.title('Amplified Input Image with amp = {}'.format(int(avg_amp[index])))
#     plt.axis('off')
#     plt.subplots_adjust(wspace =0.01, hspace = 0.000001)
#     plt.show()



In [ ]:
# #visualise
# sorted_psnrs = sorted(psnrs)

# for psnr in sorted_psnrs[-5:]:
#     index = psnrs.index(psnr)
#     plt.figure(figsize=(20,12))
#     plt.subplot(2,2,1)
#     if raw:
#         plt.imshow(np.fromfile(open(os.path.join(out_dir, img_files1[index]), 'rb'), dtype=np.uint8).reshape(2160,4096), 'gray')
#     else:
#         # plt.imshow(Image.open(os.path.join(out_dir, img_files[index])), 'gray')
#         plt.imshow(gt_[index], 'gray')
#     plt.title('Ground Truth - {}'.format(img_files[index]))
#     plt.axis('off')
    
#     plt.subplot(2,2,2)
#     plt.imshow(results[index], 'gray')
#     plt.title('PSNR: '+str(round(psnr,2)) + ' SSIM: '+str(round(ssims[index], 2)))
#     plt.axis('off')
    
#     plt.subplot(2,2,3)
#     # plt.imshow(Image.open(os.path.join(out_dir, img_files[index])), 'gray')
#     plt.imshow(abs(gt_[index] - results[index]),'gray')
#     plt.title('Difference')
#     plt.axis('off')
    
#     plt.subplot(2,2,4)
#     plt.imshow(amp_imgs[index], 'gray')
#     plt.title('Amplified Input Image with amp = {}'.format(int(avg_amp[index])))
#     plt.axis('off')
#     plt.subplots_adjust(wspace =0.01, hspace = 0.000001)
#     plt.show()

